In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

deb http://packages.cloud.google.com/apt gcsfuse-focal main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1210  100  1210    0     0  28139      0 --:--:-- --:--:-- --:--:-- 28139
OK
Hit:1 http://packages.cloud.google.com/apt gcsfuse-focal InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:11

In [ ]:
!mkdir -p diffusiondb-2m-part-0001-to-0100-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-d7f74bc4b0868963a3d7c0c0d3916af3d37fb38bd3a4ddbaeeb9ea76" diffusiondb-2m-part-0001-to-0100-of-2000

2023/04/08 13:30:48.109565 Start gcsfuse/0.42.3 (Go version go1.19.5) for app "" using mount point: /content/diffusiondb-2m-part-0001-to-0100-of-2000


In [ ]:
!mkdir -p diffusiondb-2m-part-0101-to-0200-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-384f08275e3d3b1eadd281695ad754ae6f959bf91d86f82c7cfb2d73" diffusiondb-2m-part-0101-to-0200-of-2000

In [ ]:
!mkdir -p diffusiondb-2m-part-0201-to-0300-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-0201-to-0300-of-2000
!mkdir -p diffusiondb-2m-part-0301-to-0400-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-0301-to-0400-of-2000
!mkdir -p diffusiondb-2m-part-0401-to-0500-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-0401-to-0500-of-2000
!mkdir -p diffusiondb-2m-part-0501-to-0600-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-0501-to-0600-of-2000
!mkdir -p diffusiondb-2m-part-0601-to-0700-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-0601-to-0700-of-2000
!mkdir -p diffusiondb-2m-part-0701-to-0800-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-0701-to-0800-of-2000
!mkdir -p diffusiondb-2m-part-0801-to-0900-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-0801-to-0900-of-2000
!mkdir -p diffusiondb-2m-part-0901-to-1000-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-0901-to-1000-of-2000
!mkdir -p diffusiondb-2m-part-1001-to-1100-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-1001-to-1100-of-2000
!mkdir -p diffusiondb-2m-part-1101-to-1200-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-1101-to-1200-of-2000
!mkdir -p diffusiondb-2m-part-1201-to-1300-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-1201-to-1300-of-2000
!mkdir -p diffusiondb-2m-part-1301-to-1400-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-1301-to-1400-of-2000
!mkdir -p diffusiondb-2m-part-1401-to-1500-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-1401-to-1500-of-2000
!mkdir -p diffusiondb-2m-part-1501-to-1600-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-1501-to-1600-of-2000
!mkdir -p diffusiondb-2m-part-1601-to-1700-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-1601-to-1700-of-2000
!mkdir -p diffusiondb-2m-part-1701-to-1800-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-1701-to-1800-of-2000
!mkdir -p diffusiondb-2m-part-1801-to-1900-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-1801-to-1900-of-2000
!mkdir -p diffusiondb-2m-part-1901-to-2000-of-2000
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-" diffusiondb-2m-part-1901-to-2000-of-2000


In [41]:
!mkdir sentence-transformers-2.2.2
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 "kds-9ca71c17b9c7b24487321c17d4d70b79186a47e2fbed8acbf2692695" sentence-transformers-2.2.2

2023/04/08 14:24:20.571513 Start gcsfuse/0.42.3 (Go version go1.19.5) for app "" using mount point: /content/sentence-transformers-2.2.2


In [37]:
!mkdir diffusiondb-data-cleansing
!kaggle kernels output shoheiazuma/diffusiondb-data-cleansing -p diffusiondb-data-cleansing

mkdir: cannot create directory ‘diffusiondb-data-cleansing’: File exists
Output file downloaded to diffusiondb-data-cleansing/diffusiondb.csv
Kernel log downloaded to diffusiondb-data-cleansing/diffusiondb-data-cleansing.log 


Streaming output truncated to the last 5000 lines.
/kaggle/input/diffusiondb-2m-part-0001-to-0100-of-2000/a2e5d6ce-75d9-46db-9e4c-7e8e9761434b.png,2 d isometric survival rpg by vladimir kush - n 4
/kaggle/input/diffusiondb-2m-part-0001-to-0100-of-2000/12709207-fcfb-4476-969d-97f24546def6.png,"glass vodka bottle by shusei nagaoka, kaws, david rudnick, airbrush on canvas, pastel colors, cell - shaded, 8 k"
/kaggle/input/diffusiondb-2m-part-0001-to-0100-of-2000/b8f61693-4ee4-4015-8cc4-21ccd04674f2.png,"abstract glass vodka bottle by shusei nagaoka, kaws, david rudnick, airbrush on canvas, pastell colours, cell shaded, 8 k, dirty, grainy"
/kaggle/input/diffusiondb-2m-part-0001-to-0100-of-2000/dee9912b-c7b8-4682-afec-2b05513e6217.png,"a mechanical cybertronic tulip, glowing, metallic"
/kaggle/input/diffusiondb-2m-part-0001-to-0100-of-2000/fe937b9f-8c35-421a-8901-c79b86d37dac.png,painting of cadaques from vang gogh
/kaggle/input/diffusiondb-2m-part-0001-to-0100-of-2000/1e2bb099-b83f-4687-91b

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



a9-4d52-bbb4-369b7caeb854.png,an adorable cat dog fire pokemon like growlithe. very cute friendly. fluffy. beautiful. digital render.
/kaggle/input/diffusiondb-2m-part-0101-to-0200-of-2000/b3f158da-0f69-4b78-886f-8afb9c43adcb.png,neil breen as jabba the hutt
/kaggle/input/diffusiondb-2m-part-0101-to-0200-of-2000/328ee2a3-f8a9-4ce8-b886-e28bb3507e7e.png,little figure of a fox hanging at a keychain
/kaggle/input/diffusiondb-2m-part-0101-to-0200-of-2000/e2f3d31b-11c0-47e8-9516-7a5d21256c04.png,man with 3 0 fingers
/kaggle/input/diffusiondb-2m-part-0101-to-0200-of-2000/229f8a52-e61a-496c-a3f7-cd6b06e02aa5.png,saul goodman smoking in front of a gas station
/kaggle/input/diffusiondb-2m-part-0101-to-0200-of-2000/d3602c60-66a4-422e-adde-e53fcde734d5.png,joe biden in 2 8 days later movie
/kaggle/input/diffusiondb-2m-part-0101-to-0200-of-2000/b775dfdd-a4c2-41e5-b99a-66b550224a1b.png,"marvel comic book cover, cell shaded!!, hard shadows!, limited color pallet, peach black and white, drakengard, d

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



"pepe love crypto, trading, bitcoin"
/kaggle/input/diffusiondb-2m-part-1001-to-1100-of-2000/7276c68d-6e55-4c3e-9aba-a45cac9e1b18.png,trash hand drawn crying pepe very sad
/kaggle/input/diffusiondb-2m-part-1001-to-1100-of-2000/269d3558-827c-4877-9db7-57fdb78f08d1.png,meme where apes printing money
/kaggle/input/diffusiondb-2m-part-1001-to-1100-of-2000/d2854b67-b81a-4a58-b214-68bad7266193.png,"sad, furious, angry pepe, detailed"
/kaggle/input/diffusiondb-2m-part-1001-to-1100-of-2000/71369357-6981-46fb-b078-139caaa7861f.png,"real life Bert and Ernie from Sesame Street, flying a B2 bomber"
/kaggle/input/diffusiondb-2m-part-1001-to-1100-of-2000/d158726b-c774-4a54-aa55-6df46e4abacd.png,Rain world if it had the rayman origins/legends artstyle
/kaggle/input/diffusiondb-2m-part-1001-to-1100-of-2000/7032a6d2-adab-499a-a073-b0f051fdea92.png,Also kinda timings me of the artstyle of the backgrounds of badland
/kaggle/input/diffusiondb-2m-part-1001-to-1100-of-2000/92e9de82-c699-4f6e-b6cb-cdd4a75a989

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



ut/diffusiondb-2m-part-1301-to-1400-of-2000/b36b7857-b616-4190-9572-8d9d40ba6c2f.png,Soviet Poster of Walter White demading Meth from Gus
/kaggle/input/diffusiondb-2m-part-1301-to-1400-of-2000/91c491df-8e61-43c9-bb70-724d1239e89f.png,low poly red panda as a super hero saving people
/kaggle/input/diffusiondb-2m-part-1301-to-1400-of-2000/19f42d3c-1f12-4da6-a1a8-1b5663d94bf7.png,how you see yourself in human form
/kaggle/input/diffusiondb-2m-part-1301-to-1400-of-2000/4a1cd76e-6664-4d51-a46c-96164053dc24.png,"esports team logo, twitter, 4k"
/kaggle/input/diffusiondb-2m-part-1301-to-1400-of-2000/10e3bf89-572b-45af-b53e-51d05add54a9.png,"squad of us military soldiers walk through ruined city in the rain, digital art, trending on artstation, volumetric lighting, dramatic, 4k"
/kaggle/input/diffusiondb-2m-part-1301-to-1400-of-2000/46f85d89-e30e-41ac-bcea-672007ff8646.png,"human walking through the streets of a ruined city, dramatic, digital art, trending on art station, octane render, 8k, high

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Streaming output truncated to the last 5000 lines.
/kaggle/input/diffusiondb-2m-part-1601-to-1700-of-2000/f1904187-c9b5-4b7a-9096-34bb8ceff8f0.png,Nvidia GPU that has caught on fire
/kaggle/input/diffusiondb-2m-part-1601-to-1700-of-2000/3ed5908e-2b0d-4775-ab1e-3a7b6fd1246c.png,"an Nvidia GeForce RTX 3090 GPU that has caught on fire, GeForce RTX 3090 on fire"
/kaggle/input/diffusiondb-2m-part-1601-to-1700-of-2000/d8c03dc3-d899-462b-9032-f4205f6959dc.png,"official illustration of cute chibi Batman eating pizza, trending in artstation, highly detailed, award-winning, official DC comics illustration, very cute, happy, gradient background, chibi"
/kaggle/input/diffusiondb-2m-part-1601-to-1700-of-2000/52625ae3-319e-472f-bacc-c199883877ec.png,Spider-Man with his mask pulled halfway up
/kaggle/input/diffusiondb-2m-part-1601-to-1700-of-2000/30898d6e-5ac0-4042-ba17-3722b1d6b3c4.png,Mark Grayson from Invincible eating Pizza
/kaggle/input/diffusiondb-2m-part-1601-to-1700-of-2000/62d8c74b-6c9f-4a31

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
import os
import json
from google.colab import drive

!pip install kaggle

drive.mount('/content/gdrive')

f = open("/content/gdrive/My Drive/Colab Notebooks/Kaggle/kaggle.json", 'r')
json_data = json.load(f) 
os.environ['KAGGLE_USERNAME'] = json_data['username']
os.environ['KAGGLE_KEY'] = json_data['key']

!kaggle competitions download -c stable-diffusion-image-to-prompts
!mkdir /content/input
!mv /content/stable-diffusion-image-to-prompts.zip /content/input/stable-diffusion-image-to-prompts.zip
!unzip /content/input/stable-diffusion-image-to-prompts.zip -d /content/input/

Archive:  /content/input/stable-diffusion-image-to-prompts.zip
  inflating: /content/input/images/20057f34d.png  
  inflating: /content/input/images/227ef0887.png  
  inflating: /content/input/images/92e911621.png  
  inflating: /content/input/images/a4e1c55a9.png  
  inflating: /content/input/images/c98f79f71.png  
  inflating: /content/input/images/d8edf2e40.png  
  inflating: /content/input/images/f27825b2c.png  
  inflating: /content/input/prompts.csv  
  inflating: /content/input/sample_submission.csv  


# Library

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from PIL import Image
from tqdm.notebook import tqdm
from scipy import spatial
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import CosineAnnealingLR
from torchvision import transforms
import timm
from timm.utils import AverageMeter
import sys
sys.path.append('../input/sentence-transformers-222/sentence-transformers')
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

# Config

In [ ]:
class CFG:
    model_name = 'vit_base_patch16_224'
    input_size = 224
    batch_size = 64
    num_epochs = 6
    lr = 1e-4
    seed = 42

In [ ]:
def seed_everything(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed)
        torch.backends.cudnn.deterministic = True


seed_everything(CFG.seed)

# Dataset

In [ ]:
class DiffusionDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = Image.open(row['filepath'])
        image = self.transform(image)
        prompt = row['prompt']
        return image, prompt


class DiffusionCollator:
    def __init__(self):
        self.st_model = SentenceTransformer(
            '/kaggle/input/sentence-transformers-222/all-MiniLM-L6-v2',
            device='cpu'
        )
    
    def __call__(self, batch):
        images, prompts = zip(*batch)
        images = torch.stack(images)
        prompt_embeddings = self.st_model.encode(
            prompts, 
            show_progress_bar=False, 
            convert_to_tensor=True
        )
        return images, prompt_embeddings
    
    
def get_dataloaders(
    trn_df,
    val_df,
    input_size,
    batch_size
):
    transform = transforms.Compose([
        transforms.Resize(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])

    trn_dataset = DiffusionDataset(trn_df, transform)
    val_dataset = DiffusionDataset(val_df, transform)
    collator = DiffusionCollator()
    
    dataloaders = {}
    dataloaders['train'] = DataLoader(
        dataset=trn_dataset,
        shuffle=True,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=2,
        drop_last=True,
        collate_fn=collator
    )
    dataloaders['val'] = DataLoader(
        dataset=val_dataset,
        shuffle=False,
        batch_size=batch_size,
        pin_memory=True,
        num_workers=2,
        drop_last=False,
        collate_fn=collator
    )
    return dataloaders

# Train

In [ ]:
def cosine_similarity(y_trues, y_preds):
    return np.mean([
        1 - spatial.distance.cosine(y_true, y_pred) 
        for y_true, y_pred in zip(y_trues, y_preds)
    ])

In [ ]:
def train(
    trn_df,
    val_df,
    model_name,
    input_size,
    batch_size,
    num_epochs,
    lr
):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    dataloaders = get_dataloaders(
        trn_df,
        val_df,
        input_size,
        batch_size
    )

    model = timm.create_model(
        model_name,
        pretrained=True,
        num_classes=384
    )
    model.set_grad_checkpointing()
    model.to(device)
    
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr)

    ttl_iters = num_epochs * len(dataloaders['train'])
    scheduler = CosineAnnealingLR(optimizer, T_max=ttl_iters, eta_min=1e-6)
    criterion = nn.CosineEmbeddingLoss()
    
    best_score = -1.0

    for epoch in range(num_epochs):
        train_meters = {
            'loss': AverageMeter(),
            'cos': AverageMeter(),
        }
        model.train()
        for X, y in tqdm(dataloaders['train'], leave=False):
            X, y = X.to(device), y.to(device)

            optimizer.zero_grad()
            X_out = model(X)
            target = torch.ones(X.size(0)).to(device)
            loss = criterion(X_out, y, target)
            loss.backward()

            optimizer.step()
            scheduler.step()

            trn_loss = loss.item()
            trn_cos = cosine_similarity(
                X_out.detach().cpu().numpy(), 
                y.detach().cpu().numpy()
            )

            train_meters['loss'].update(trn_loss, n=X.size(0))
            train_meters['cos'].update(trn_cos, n=X.size(0))

        print('Epoch {:d} / trn/loss={:.4f}, trn/cos={:.4f}'.format(
            epoch + 1,
            train_meters['loss'].avg,
            train_meters['cos'].avg))

        val_meters = {
            'loss': AverageMeter(),
            'cos': AverageMeter(),
        }
        model.eval()
        for X, y in tqdm(dataloaders['val'], leave=False):
            X, y = X.to(device), y.to(device)

            with torch.no_grad():
                X_out = model(X)
                target = torch.ones(X.size(0)).to(device)
                loss = criterion(X_out, y, target)

                val_loss = loss.item()
                val_cos = cosine_similarity(
                    X_out.detach().cpu().numpy(), 
                    y.detach().cpu().numpy()
                )

            val_meters['loss'].update(val_loss, n=X.size(0))
            val_meters['cos'].update(val_cos, n=X.size(0))

        print('Epoch {:d} / val/loss={:.4f}, val/cos={:.4f}'.format(
            epoch + 1,
            val_meters['loss'].avg,
            val_meters['cos'].avg))
        
        if val_meters['cos'].avg > best_score:
            best_score = val_meters['cos'].avg
            torch.save(model.state_dict(), f'{model_name}.pth')

In [ ]:
df = pd.read_csv('/kaggle/input/diffusiondb-data-cleansing/diffusiondb.csv')
trn_df, val_df = train_test_split(df, test_size=0.1, random_state=CFG.seed)

In [ ]:
# trn_df = df[:2]
# val_df = df[2:4]
# val_df

In [ ]:
train(trn_df, val_df, CFG.model_name, CFG.input_size, CFG.batch_size, CFG.num_epochs, CFG.lr)

  0%|          | 0/2170 [00:00<?, ?it/s]

Epoch 1 / trn/loss=0.5024, trn/cos=0.4976


  0%|          | 0/242 [00:00<?, ?it/s]

Epoch 1 / val/loss=0.4674, val/cos=0.5326


  0%|          | 0/2170 [00:00<?, ?it/s]

...

Epoch 5 / val/loss=0.4456, val/cos=0.5544


  0%|          | 0/2170 [00:00<?, ?it/s]

Epoch 6 / trn/loss=0.2856, trn/cos=0.7144


  0%|          | 0/242 [00:00<?, ?it/s]

Epoch 6 / val/loss=0.4529, val/cos=0.5471


In [ ]:
# model = timm.create_model(
#     CFG.model_name,
#     pretrained=True,
#     num_classes=384
# )
# model.load_state_dict(torch.load('{CFG.model_name}.pth'))

# model.eval()
# for X, y in tqdm(dataloaders['val'], leave=False):
#     X, y = X.to(device), y.to(device)

#     with torch.no_grad():
#         X_out = model(X)
#         target = torch.ones(X.size(0)).to(device)
#         loss = criterion(X_out, y, target)
